In [3]:
import torch 
import torchvision

torch.__version__, torchvision.__version__

('2.4.0', '0.19.0')

In [18]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [23]:
def set_seed(seed: int):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

In [13]:
from pathlib import Path 

IMAGE_PATH = Path("data/pizza_steak_sushi")
train_dir = IMAGE_PATH / "train"
test_dir = IMAGE_PATH / "test"

In [14]:
from torchvision import transforms 
from going_modular.going_modular import data_setup

normalize = transforms.Normalize(
        mean=[0.485, 0.456, 0.406], 
        std=[0.229, 0.224, 0.225]
)

manual_transform = transforms.Compose([
    transforms.Resize(size=(224, 224)),
    transforms.ToTensor(), 
    normalize
])

train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(train_dir, test_dir, manual_transform, 32, 0)

In [16]:
class_names

['pizza', 'steak', 'sushi']

In [19]:
from transfer_learning import model_builder

model_0 = model_builder.create_model_baseline_effnetb0(out_feats=len(class_names), device=device)

In [21]:
from torchinfo import summary

summary(model_0, (32, 3, 224, 224), col_names=["input_size", "output_size", "num_params", "trainable"],)

Layer (type:depth-idx)                                  Input Shape               Output Shape              Param #                   Trainable
EfficientNet                                            [32, 3, 224, 224]         [32, 3]                   --                        Partial
├─Sequential: 1-1                                       [32, 3, 224, 224]         [32, 1280, 7, 7]          --                        False
│    └─Conv2dNormActivation: 2-1                        [32, 3, 224, 224]         [32, 32, 112, 112]        --                        False
│    │    └─Conv2d: 3-1                                 [32, 3, 224, 224]         [32, 32, 112, 112]        (864)                     False
│    │    └─BatchNorm2d: 3-2                            [32, 32, 112, 112]        [32, 32, 112, 112]        (64)                      False
│    │    └─SiLU: 3-3                                   [32, 32, 112, 112]        [32, 32, 112, 112]        --                        --
│    └─Sequential

In [22]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model_0.parameters(), lr=0.001)

In [33]:
from transfer_learning import engine 

set_seed(seed=42)
result = engine.train(model=model_0, train_dataloader=train_dataloader, test_dataloader=test_dataloader, loss_fn=loss_fn, optimizer=optimizer, device=device, epochs=10)

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 0.2774 | train_acc: 0.9297 | test_loss: 0.3104 | test_acc: 0.8968


 10%|█         | 1/10 [00:03<00:33,  3.73s/it]

Epoch: 2 | train_loss: 0.2341 | train_acc: 0.9648 | test_loss: 0.3130 | test_acc: 0.9280


 20%|██        | 2/10 [00:07<00:29,  3.72s/it]

...

 80%|████████  | 8/10 [00:28<00:07,  3.51s/it]

Epoch: 9 | train_loss: 0.2650 | train_acc: 0.9922 | test_loss: 0.3288 | test_acc: 0.9176


 90%|█████████ | 9/10 [00:32<00:03,  3.56s/it]

Epoch: 10 | train_loss: 0.2131 | train_acc: 0.9922 | test_loss: 0.2902 | test_acc: 0.9176


100%|██████████| 10/10 [00:35<00:00,  3.56s/it]


In [26]:
result

{'train_loss': [1.0900676250457764,
  0.9045843482017517,
  0.8018361926078796,
  0.6778495907783508,
  0.7158164978027344],
 'train_acc': [0.4375, 0.65234375, 0.73046875, 0.859375, 0.69921875],
 'test_loss': [0.9141465425491333,
  0.8036970098813375,
  0.6782492399215698,
  0.6665419141451517,
  0.662106990814209],
 'test_acc': [0.5085227272727273,
  0.7537878787878788,
  0.8967803030303031,
  0.7945075757575758,
  0.7945075757575758]}